# This notebook is used as example to provide quick and direct predictions on single or a few molecules. 

#### 1. Go to the directory where scripts are saved. 

In [1]:
# you may change this directory
cd /scratch/dz1061/packages/solvation_energy_prediction/scr

/scratch/dz1061/packages/solvation_energy_prediction/scr


In [3]:
from helper import *
from data_process import *
from trainer import *
from model import *
from layers import *
from gen_conf import *
from utils import *

#### 2. Starting with SMILES for one given molecule, let's first generate a conformer for it. 

In [4]:
mol = Chem.MolFromSmiles('CCc1cnccn1')

In [5]:
# the genrator function and related functions are saved in gen_conf.py. It will generate a sdf and xyz file.
# example_1 is the name you give to the example molecule. 
# path is where sdf and xyz files are saved. You may provide a specific directory. 
name = 'example_1' # you may change this name
file_path = '/scratch/dz1061/packages/solvation_energy_prediction/results/Direct_test' # you may change this directory
generator(mol, name, file_path)

#### 3. Then we generate initial features for the given molecule by using sdf and xyz file.

In [6]:
example_graph = generate_graphs(os.path.join(file_path, name+'.sdf'), os.path.join(file_path, name+'.xyz'))

Take a quick look at the attributes we need for a given molecule. 

In [7]:
example_graph

{'x': tensor([[4.6251, 3.1136, 2.6679,  ..., 0.0000, 0.0000, 0.0000],
         [4.7142, 2.3933, 3.2023,  ..., 0.0000, 0.0000, 0.0000],
         [4.3858, 0.6299, 2.8670,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [2.1847, 0.0665, 0.6709,  ..., 0.0000, 0.0000, 0.0000],
         [0.8580, 0.0625, 0.4805,  ..., 0.0000, 0.0000, 0.0000],
         [0.9875, 0.0625, 0.4784,  ..., 0.0000, 0.0000, 0.0000]]),
 'Z': tensor([6, 6, 6, 6, 7, 6, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1]),
 'N': tensor([16.]),
 'edge_attr': tensor([[0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0

Let's save it for later use.

In [8]:
# you need to manually create a folder called raw under file_path.
torch.save([example_graph], '/scratch/dz1061/packages/solvation_energy_prediction/results/Direct_test/raw/temp.pt')

#### 4. Generate a data loader for the given molecule.

In [9]:
example_dataset = GraphDataset_test('/scratch/dz1061/packages/solvation_energy_prediction/results/Direct_test')

Processing...
Done!


In [10]:
example_loader = DataLoader(example_dataset, batch_size=1, num_workers=0)

#### 5. Load model weights from trained model. 

In [11]:
model_path = '/scratch/dz1061/packages/solvation_energy_prediction/models/FreeSolv'

In [15]:
# load in configuration file
config = loadConfig(os.path.join(model_path))
config_all = {**config['data_config'], **config['model_config'], **config['train_config']}
config_all['deg'] = torch.tensor([0, 5351, 348, 1867, 1565, 0])
config_all['normalize'] = True
config_all['energy_shift'] = torch.tensor([config_all['energy_shift_value']])
config_all['energy_scale'] = torch.tensor([config_all['energy_scale_value']])
config_all['device'] = 'cpu'

# building models
model = get_model(config_all)
state_dic = torch.load(os.path.join(model_path, 'model_best.pt'), map_location=torch.device('cpu'))
model.load_state_dict(state_dic)
model.eval()
    
with torch.no_grad():
    for data in example_loader:
        print(model(data))

tensor([-5.6809])
